## Topic Modeling — With Tomotopy

An attempt of topic modeling in the american women corpus of correspondance of the 18th Centrury

### Import Pachages

In [1]:
import tomotopy as tp
import little_mallet_wrapper
import seaborn
import glob
from pathlib import Path
import pandas as pd

In [2]:
file = open('women_corpus.txt', encoding = 'utf-8').read()

In [3]:
training_data = []
original_texts = []
titles = []


processed_text = little_mallet_wrapper.process_string(file, numbers='remove')
training_data.append(processed_text)
original_texts.append(file)
titles.append(Path(file).stem)

In [4]:
len(training_data), len(original_texts), len(titles)

(1, 1, 1)

## Train Topic Model
We’re going to train our topic model with the tp.LDAModel() function, which provides Latent Dirichlet Allocation(LDA) topic modelling. Its implementation is based on following papers:

* Blei, D.M., Ng, A.Y., &Jordan, M.I. (2003). Latent dirichlet allocation.Journal of machine Learning research, 3(Jan), 993 - 1022.
* Newman, D., Asuncion, A., Smyth, P., &Welling, M. (2009). Distributed algorithms for topic models. Journal of Machine Learning Research, 10(Aug), 1801 - 1828.

In [7]:
# Number of topics to return
num_topics = 10
# Numer of topic words to print out
num_topic_words = 5

# Intialize the model
model = tp.LDAModel(k=num_topics)

# Add each document to the model, after splitting it up into words
for text in training_data:
    model.add_doc(text.strip().split())

print("Topic Model Training...\n\n")
# Iterate over the data 10 times
iterations = 10
for i in range(0, 100, iterations):
    model.train(iterations)
    print(f'Iteration: {i}\tLog-likelihood: {model.ll_per_word}')


Topic Model Training...


Iteration: 0	Log-likelihood: -10.234096369451956
Iteration: 10	Log-likelihood: -9.997525888857785
Iteration: 20	Log-likelihood: -9.853015554279187
Iteration: 30	Log-likelihood: -9.748333314132855
Iteration: 40	Log-likelihood: -9.669984987815438
Iteration: 50	Log-likelihood: -9.605346027337633
Iteration: 60	Log-likelihood: -9.551387727581076
Iteration: 70	Log-likelihood: -9.509453644633965
Iteration: 80	Log-likelihood: -9.463135145195055
Iteration: 90	Log-likelihood: -9.430732217694144


## Display Topics and Top Words
To examine the 15 topics that the topic model extracted from theWomen Correspondance Corpuss, run the cell below.

Take a minute to read through every topic. Reflect on what each topic seems to capture as well as how well you think the topics capture the broad themes of the entire collection. Note any oddities, outliers, or inconsistencies.

In [8]:
# Print out top 10 words for each topic
print("\nTopic Model Results:\n\n")

topics = []
topic_individual_words = []
for topic_number in range(0, num_topics):
    topic_words = ' '.join(word for word, prob in model.get_topic_words(topic_id=topic_number, top_n=num_topic_words))
    topics.append(topic_words)
    topic_individual_words.append(topic_words.split())
    print(f"✨Topic {topic_number}✨\n\n{topic_words}\n")


Topic Model Results:


✨Topic 0✨

much well write long return

✨Topic 1✨

ever much may would upon

✨Topic 2✨

dear letter sir one time

✨Topic 3✨

since dear country que hope

✨Topic 4✨

mrs time may better dear

✨Topic 5✨

shall dear sister one many

✨Topic 6✨

much know may letter great

✨Topic 7✨

upon would must letter see

✨Topic 8✨

would present little upon cannot

✨Topic 9✨

adams may think must sent



In [11]:
topic_distributions = [list(doc.get_topic_dist()) for doc in model.docs]
topic_distributions

[[0.06715536,
  0.07019223,
  0.07490364,
  0.07149399,
  0.05257038,
  0.07148009,
  0.06316005,
  0.06420261,
  0.05014551,
  0.07439326,
  0.0646514,
  0.060446747,
  0.05241664,
  0.07357769,
  0.08921038]]

In [12]:
from IPython.display import Markdown, display
import re

def make_md(string):
    display(Markdown(str(string)))

def get_top_docs(docs, topic_distributions, topic_index, n=5):

    sorted_data = sorted([(_distribution[topic_index], _document)
                          for _distribution, _document
                          in zip(topic_distributions, docs)], reverse=True)

    topic_words = topics[topic_index]

    make_md(f"### ✨Topic {topic_index}✨\n\n{topic_words}\n\n")
    print("---")

    for probability, doc in sorted_data[:n]:
        # Make topic words bolded
        for word in topic_words.split():
            if word in doc.lower():
                doc = re.sub(f"\\b{word}\\b", f"**{word}**", doc, re.IGNORECASE)

        make_md(f'✨  \n**Topic Probability**: {probability}  \n**Document**: {doc}\n\n')

    return

In [13]:
# Display 6 documents for topic 0
get_top_docs(titles, topic_distributions, topic_index=0, n=6)

### ✨Topic 0✨

better much one president town think happy give mrs sure



---


✨  
**Topic Probability**: 0.06715536117553711  
**Document**:  for his Shear of Tobaco at Linchburge. Mr. Lewis hopes to go to Bedford to the Next Court. We have never been to Montello; inten’d our selves that pleasure yesterday, but was prevented by the weather which is yet wet with us. George has been frequently hear and reports that all is well. We have been sickly genraly and now have the Measels and hooping cough in the Neibourhod. The Measels is a moung your Negros at Bedford but thay have lost none. Hope you will excues this ill writen scroll as Mr. Lewis attemt’d and could not finish his litter. I am **much** hurry’d with this. Mr. Lewis desiers me to tender his most respectfull Compliments. I am Sir with very Great Esteem Your Sincear Friend, M. Lewis
Dear Papa Eppington july 20, [1790] I hope you will excuse my not writing to you before tho I have none for myself I am very sorry to hear that you have been sick but flatter myself that it is over. My aunt skipwith has been very sick but she is **better** now. We have been to see her two or three times. You tell me in your last letter that you will see me in september but I have received a letter from my Brother that says you will not be here before February. As his is later than yours I am afraid you have changed your mind. The books that you have promised me are Anacharses and gibbons roman empire. If you are coming in september I hope you will not forget your promise of bying new jacks for the pianeforte that is at monticello. Adieu my dear Papa. I am your affectionate Daughter Mary Jefferson My Uncle and Aunt desire to be affectionately remembered to you. M.J.
Dear Sir London July the 20th 1790. I have the pleasure of acquainting your Excellency that the Queen of Portugal has appointed Monsr. le Chevr. de Freire to be her Minister to America. This Gentleman was to have gone to another Court, but Monsr. le Chevr. de Pinto, knowing that Monsr. de Freire had been imployed here to negotiate the Treaty between Portugal, and America, and also, as he was personally acquainted with your Excellency, and Our Vice President Mr. Adams, and understood perfectly the English Language, the Queen altered her first intention, and appointed him immediately as her Minister to our States. You will find him worthy of every attention you will be pleased to shew him, as a private Gentleman, and also as a Minister. He is possesed of every Virtue which constitutes a truly great and good Man. It will make him **happy** to be upon the strictest friendship with you. He has been Three times Chargé des affair’s here. He is greatly beloved and respected by all his acquaintance. He is extremely sorry to leave this Country. I have told him, I hope he will find America and the Americans in every respect as agreeable as the people of this Country. I have taken the liberty to tell him, I am certain you and Mr. Adams will Make it a point to contrive to Make him pass his time in the Most agreeable Manner. He would be **happy** to know what things he Must bring, or if it would be **better** for him to buy them in America. Being Unacquainted with the business, I told him, I would take the liberty to trouble you. It is as followes, whether he can find a House ready furnished, and if not furnished, would you advise him to buy his furniture here or in America. There being numberless things necessary for a Minister which I am a total Stranger to you will add greatly to the many obligations you have already confered upon Me to send Me a particular account of every thing I have desired. I have heard from so many persons that My dearest Friend Miss Jefferson has changed her Name, that, I **think**, Mr. Paradise and Myself may join in congratulating you on the event, which we sincerely pray may make you the happiest of Fathers. We have heard that her Sister lives with her at your Seat in Virginia. We must trouble you to present our Congratulations to Mr. and Mrs. Randolph and the now Miss Jefferson, and assure them, that we often, and often, talk of them, and **think** of the happy Days, we passed with them. On Sunday July the 3d, arrived in this City Mrs. Barlow  the Wife of Mr. Barlow. I was in hopes she would have brought us Letters from Our Friends at New York, but great was our Uneasiness when she said she had none. Mrs. Blagden is come from France to Conduct her to her Husband. My Daughter in her last Letter acquaints me she had Answered your Excellencies Letter, and was affraid, as she had not heard again from you, You had Not received her Answer. I did not know Your Excellency had ever written to her. This puts Me under another obligation, and for which I beg you will accept My thanks. Dr. Bancroft is still the same, good, and kind friend. Mr. P. is perfectly sober, and is in perfect health and should be happy to be employed by his Country to which he now belongs as a Citizen. All Europe is in a great Bustle, France seems in My humble opinion to be the Most wise in every respect. My wish is that they would stop, and go no further, as things over done is generally spoilt. We beg you will present our respectful Compliments to our President his Lady and family and our Compliments to our Vice President and his Lady and Family and to all our Friends both Ladies and Gentlemen. Mr. Paradise and Myself join in Affectionate and Grateful Compliments and a Thousand Thanks to your Excellency for all your favours. Dear Sir I have the Honour to be your ever Affect. Humble. Servant Lucy Paradise I shall be greatly obliged to you Sir, if you will be so good as to send the inclosed Letter to my Cousin. P.S. I am told Lord Hawksbury is to be the Minister for the foreign department in the room of the Duke of Leeds who it is said will resign.
23 juillet Je vous ai ecrit en octobre, vous me repondez le 2 avril, et je recois votre lettre le 20 juillet. Autant vaudroit dater des champs elisées que de New york. Ah, je l’avois bien dit que je ne vous reverrois jamais; mes adieux ont devances les votres, et lorsque je fus chez vous apres le depart, ma tristesse extreme fut un pressentiment bien juste. Enfin soyez heureux, comme vous lentendez, a votre maniere, bien loin de moi, separé meme de vos filles. Il est donc bien vrai que je ne vous reverrai jamais. Je nay dautre affaire que celle daimer et de regretter mes amis; ainsi, mon souvenir vous est assuré, mais le vôtre, ah, quil est douteux.—Je vais vous predire votre sort, vous vous remarirez, oui, c’est sur, Votre femme sera heureuse et vous aussi, je lespere bien; mais qui me retracera a votre souvenir? Allons, encore un sacrifice a la necessité. Votre fille est donc mariée, et son gout s’est raporter avec le votre. Les bons peres doivent etre ainsi recompensé par leurs enfans. Et cette chere Pauly qui parle de moi, pourquoi navez vous pas eu assez de confiance pour me la laisser? Mais non, il eut ete trop difficile de vous la rendre. Faitte mille amitie a ces deux chere filles. Je parie que laînée a deja sa jolie taille gâté.—Mr. Short crain de quitter paris. Lhistoire de notre revolution est interessante a suivre, quoique tout les gens sensés trouvent que depuis longtems nous avons passé le but. Lordre judiciaire et le militaire ne sont pas encor organisé; la municipalite de paris n’est encor que provisoire. Et pourtant lon a fait beaucoup de choses: en destruction il ny a plus rien a inventer. Loperation sur le clerge est un bien pour le siecle a venir mais le present est dur a suporter. Mr. Neker est perdu aux yeux de la multitude. Le 12 juillet de cette année, juste le bout de l’an ou Msrs. les duc dorleans et Neker ont ete portes comme idole du peuple, ce même peuple les insulte et les place dans la boue. Quelle versatilité dans lopinion et que pour longtems encor les bonnes gens aimeront a vivre ignoré et loin de la Capitale.—Nous avons fait choix dune abbaye fort simple, fort isolée, pour lachetter, mais jay des concurrens et je crains de ne pas lobtenir.—Nous fessons tout ce que nous pouvons pour subsister et croire aux assignats, le numeraire disparoit largent sachette un prix fou.—Ah pourquoi un si grand espace nous separe til? Jaimerois a chercher près de vous un repos après lequel je soupire. Mr. de Corny aime votre pays, il forme bien des voeux.—Mais il faudroit realiser sa fortune, ma jolie maison est a vendre mais on ne veut pas lachetter. Je ne puis concevoir qui arrette Mde. De brean [Brehan]. Je voudrois être a sa place, je vous lassure.—Mr. de Corny a bien souffert pour sa santé, il ma donné des inquiétudes très vives. Le malheur la vieillesse se peignoient sur son visage. Je le trouve mieux. On na pas encor remboursé sa charge et on ne luy en paye pas même les interets. Degoutez bien toute vos dames de venir en France et assurez les que leur imagination peut seul embellir paris quil ne faut plus venir chercher. Mr. de Corny est penetre destime pour vous, il vous regrette, et pour luy, et pour moi. Donnez moi vos commissions, donnez en a Mr. de Corny. Lon disoit lautre jour que l’assemblée nationale ne vouloit plus dambassadeurs. Les Cours etrangeres suivroient ce même exemple. Alors chacun naura plus quun agent corresponde[nt]. Si ce roman a lieu, songez a Mr. de Corny. Il aime a soccuper  et je le crois plus malheureux de navoir rien a faire que dêtre presque ruiné. Mr. Short me donne quelquefois de vos nouvelles. Je voudrois qu’il fut possible de ne pas le remplacer. Il me semble qu’il méritte dêtre bien traité. Mais, sil en est autrement, ne madressez pas celuy qui vous succedera. Je crois quil me feroit de la peine de le voir. Mr. de la fayette justifie bien ladoption quon en a fait en amerique. Cest une prudence, une tranquilité, une perseverance qui le font adorer. Je crois quil ne peut y avoir que les mechans qui [luy] refusent justice. Adieu mon cher Monsieur croyez a mes sentiments pour vous, ils sont inalterables. Mr. Church est membre du parlement Angelique se porte très bien. Kitty ne mécrit plus. Mais l’essentiel, cest que sa mère soit contente delle et elle l’est.
Monsieur et respectable ami (agrées ce titre) ce 25 juillet 1790 J’ay étté bien charmée de recevoir de vous une lettre. Impatiente d’apprendre de vos nouvelles, j’en croyois le moment passé. Je croyois qu’un si long trajet de Mer et des objets si agréables pour vous à retrouver avoient effacé quelques sentimens flatteurs pour moi, s’ils avoient jamais existé. Mon Cousin m’ayant écrit au mois de decembre m’apprit les détails de votre arrivée, de laquelle Monsr. Short ne m’avoit dit que votre débarquement en bonne santé. Cette lettre m’apprit aussi les attentions pleines de bonté que vous eûtes pour lui et sa femme. Aucun détail du premier diner qu’il put avoir l’honneur d’aller faire chés vous ne lui avoit échapé. Le soin que vous prîtes de lui cacher le refroidissement que je devois ressentir de son omission longue à m’écrire le touchoit beaucoup, il m’a touché également. Vous presumiés, Monsieur, que mon amitié pour lui se retrouveroit la même lorsque j’apprendrois qu’il n’y avoit pas de sa faute. J’ay jugé par les expressions de sa lettre qu’elle n’avoit pas  du être la seule qu’il m’eut écritte, mais je n’en ai pas recu d’autre. Vous ne voulés pas que nous nous haissions, et qui y a-t-il de plus propre à me le rendre interessant que tout le bien que vous avés la bonté de m’en dire. Un éloge de votre part a bien plus de valeur que ceux qu’on entend faire ordinairement. Les marques d’estime dont vous l’honnorés, Monsieur, sont bien propres à lui donner de la considération parmi ses voisins, ses bons voisins qui sont si honnêtes que de me savoir gré d’avoir fait du bien à mon cousin. Cela prouve leur amour pour la bienfaisance. Dans la lettre que vous m’avés fait l’honneur de m’ecrire, il m’a paru que vous croyiés que j’ay mis à fond perdu la pareille somme que je lui ai donnée: je ne l’ai pas fait et je ne puis guère, quelque envie que j’aye de lui procurer de l’aisance, penser à le faire, à cause de l’incertitude de touttes les fortunes en France. Je crois que les finances d’Amérique vont prendre une stabilité qui sera l’effet de la bonne constitution que tous les Etats Unis ont adoptée maintenant. Je verrois avec confiance mon Cousin y mettre des sommes, elles lui seroient d’un plus grand rapport qu’ici, mais je n’ai pas même pu encore retirer mon bien de la main des héritiers de feu mon masi. J’ai avec cela un autre Cousin qui est son frere qui n’a pas moins besoin des mêmes secours. Ces deux frères s’aiment et le vôtre croit sans sujet que son frère de la Martinique va venir le joindre. J’ay appris avec beaucoup de joye le mariage de Mademoiselle Jefferson; recevés en, je vous prie, Monsieur, mon Compliment sincère. Permettés aussi qu’en lui faisant pareillement compliment je lui souhaitte tout le bonheur dont on peut jouir dans ce lien quand on se convient bien. Mon Cousin, à qui vous avés fait l’honneur d’être de la Noce, m’en a fait part. Il y avoit un peu de tems lorsque j’ay recu votre lettre, Monsieur, que [je] savois votre acception de la place où votre mérite vous a appelé. Ce qui est un bien pour votre Païs est une perte pour celui ci, et particulierement pour ceux qui avoient l’avantage d’être en connoissance et en societté avec vous. Il n’y a personne qui le sente plus que moi. Je devois vous revoir et vous m’ecrivés une lettre d’adieux. Ah, quelle triste chose que ce mot! Quoy vous avés la bonté de priser tant les témoignages de consideration et d’attachement que je vous ay donnés pour me dire que vous en ressouviendrés toutte votre vie, ne pourrés vous plus me donner de vos nouvelles si chères? Ce sera la preuve que vous me tiendrés la parole de ne point m’oublier. Vous me souhaités une longue vie; il ne faut pas faire ce souhait aux personnes malheureuses. Pour vous, dont la  conservation est véritablement précieuse, avés vous assés soin de votre santé? Ce cruel mal de tête vous est donc revenu et vous a causé une maladie grave. Croyés plus à la Medecine. Vous avés, dit-on, de bons medecins. N’omettés rien de ce qu’il faut pour prevenir le mal. Vous êtes, je crois, assés informé de tous les événemens de notre révolution pour que je pense à vous en instruire. Je vous dirai seulement que je vois avec satisfaction notre constitution se consolider. La grande conféderation qui vient de se faire a présenté aux yeux un grand spectacle et à la raison de grands sujets de joye et de securitté pour l’avenir. La lecture que je viens de faire des observations sur la vente des biens Nationaux par Mr. Le Couteux et sur la dette publique me rassurent aussi beaucoup sur la prétendue banqueroutte. Il n’y a que les ennemis de la revolution, qui malheureusement sont en grand nombre, qui cherchent les moyens de la faire faire en l’annonçant. On va faire ici en Public l’eloge funeraire de Monsr. Franclin. Puissent les grands hommes qui comme lui ont eu part à l’heureuse révolution d’Amérique faire longtems notre admiration et ne nous mettent en deuil qu’après un tems bien eloigné. Je vous prie de recevoir les assurances des sentimens pleins d’un tendre attachement ainsi que d’une considération distinguée avec lesquels j’ay l’honneur d’etre, Monsieur Votre tres humble et tres obeissante servante Plumard Bellanger
Dear Sir Eppington August 11. 1790. With pleasure would we embrace your kind invitation to Monticellow if it was possible, but Mr. Eppes will at that time be so engag’d in manufacturing, it will be impossible to leave home tho’ flatter our selves we shall be favour’d with your company at Eppington. Your compliment with regard to dear Polly, I assure you would be no small inducement (tho’ I am assured your own judgment is quite sufficient) for I have every thing **much** at heart that can tend in the smalest degree to her happiness. She has lost a  month in consequence of my attending on my poor Sister Skipwith who has been very near bidding us a final adieu with a nervous fever but, thank God she is now doing very well.-I have the highest sense of your attentions to both the dear girls and am well assur’d they have only to communicate their wants, and you will supply them. Your plan of fixing dear Patsy in Albemarle were you to be there your self I should wish to take place, as I would not deprive you of the happiness you must have in such a daughters being near you, but as I suppose that will not be the case, must confess from selfish views I can not be pleased with it as I flatter my self we shall now see her frequently, tho’ she has set out but badly. We have been favour’d with her company only once. We had the pleasure to hear she was well last week. I am sorry to hear you have been so much afflicted with the head ache and sincerely hope the pure air of Monticellow will put an end to it. Yours with sincere friendship, E Eppes
Dear Papa [ca. 11 Aug. 1790] I have just received your last favour july 25 and am determined to write to you every day till I have discharged my debt. When we  were in Cumberland we went to Church and heard some singing Masters that sung verywell. They are to come here to learn my cousins to sing and as I know you have no objections to my learning any thing I am to be a scholar and hope to **give** you the pleasure of hearing an anthem. We had pease the 14 of may and strawberries the 17 of the same month tho not in that abundance we are accustomed to in consequence of a frost this spring. As for the martins swallows and whippoorwills I was so taken up with my chickens that I never attended to them and therefore cannot tell you when they came tho I was so unfortunate as to lose half of them for my cousin Bolling and myself have raised but 13 between us. Adieu my Dear Papa believe me to be your affectionate daughter, Mary Jefferson
Dear Sir London Sept. the 1st. 1790 I have done Myself the honour of writing to your Excellency by Mr. Broom. The purpot of that Letter was to acquaint you that the Queen of Portugal had appointed Chevr. de Freire to be her Minister to our States. He was to have gone to a Court in Europe. But Monsr. le Chevr. de Pinto knowing Mr. Freire knew more of Amerca then, perhaps, any of his Countrymen, and also, as he had the honour of knowing you, and Mr. Adams personally, the Queen altered her first intentions, and appointed him Minister to America. I have assured Mr. Freire he will be more pleased with America, and the Americans then he at present supposes. I have taken the liberty to tell him, I make no doubt he will be pleased with the attentions he will meet with from your Ecellencies. I hope you have received all the Money you had the goodness to lend Mr. Paradise during his stay at Paris. I should be obliged to your Excellency to let me know. The beginging of November we shall  quit our house in London, as it is too dear a place for Us to live in. I believe we shall go to Bath to Lodge and Board for a few Months. Therefore I shall esteem Myself honoured if you would be please to write to Me and direct your Letter for Us to Mr. Anderson our Merchant and put them under cover to him and desire him to be particular to deliver your Letter safe to Us. Dr. Bancroft is the Same good and true friend. Indeed, I do not know what I should do, if I had not him to assist Me. I wish greatly to return home, but, I fear, I may wish long enough for until all the Debts are paid here I must stay. I have told Mr. P I wish to go and see America once More before I get too old and that I would return to him again. Indeed, I think it necessary. The longest time I wish to stay is **one** year. Will you My dear Sir have the goodness to **give** Me your Advice upon the Subject? Mr. P is perfectly Sober, and well, only a little nervious owing to the heat of the Summer. He is at present in the Country for a few Days. You and your Amiable family enjoy the Blessing of health and happiness, I hope to the highest degree. I beg you to present Mr. Paradise and my affectionate Compliments to your amable family and respectful Compliments to all our Noble Friends &c &c &c &c. I remain with every sense of Gratitude Your ever Oliged, and Humble Servant and Friend, Lucy Paradise
Dear Sir London Sept. the 26th 1790. I have been honoured by your very polite and friendly Letter of June the 23d on August the 31st 1790. It brings Me an Account of My Estates &c. in Virginia. I have received your first, and Answered it. In that Letter, I trouble you with my Grateful thanks. I now repeat the Same, and wish sincerely I had it in my power by some Means to convince Your Excellency of the sincerity of my heart. I beg you will not take up your precious time in writing a third Letter on that Subject. I am perfectly sensible of the zeal you have to serve Mr. Paradise and Myself, but be assured My dear Sir, I have not the least wish to Make My appearance on  the Great and fatiguen theatre of the World. My wish is to live honestly, quiet, retired, without shew, and with all the Comforts of Life. The Person who wishes for a Publick Station ought first to know how to Make a tender affectionate Husband, Father, and Friend, and if such a person only knows the Name without the meaning, they little know their duty in a publick Life. Many who think themselves calculated for a Minister were they to be put into such a station, instead of being respected might bring upon themselves the Laughter of the whole World, which would be no pleasing sight for their near Relations. Mr. Paradise has been out of London Six Week on a visit to **one** of his Learned Friends. He writes to Me, and says he is in perfect health. At his return to London I will shew him your Excellencies kind Letter. I have found London too dear a Place to live in; therefore, I have proposed to Mr. Paradise and Dr. Bancroft, for Us to go and Lodge and Board at Bath for a little time, as I find it is the Most certain Method to live within our Income. Dr. Bancroft is always the same tender, Sincere friend. Indeed, I owe him very Great Gratitude and shall, if I live, take one of his Daughters to live with Me. This I cannot do until all the Debts are paid. I have one favour to beg of your Excellency which is to have the goodness to be My Friend and protector and to write to Me Once a Year at least. For when some persons see I am protected by My Friends in America they will be affraid (altho’ they will not say it) to use me unhandsomely for fear I should write. I earnestly beg of your Excellency to preserve Your health as it is of the greatest importance to our Country and to Me More then any American. I am happy to hear My Dear Mrs. Randolph has Married to your entire satisfaction. I hope Miss Jefferson will follow her example and that you will be blessed in the Winter of Life. Be pleased to present My affectionate Love to Mrs. Randolph and Miss Jefferson. I am with the greatest esteem Dear Sir Your Excellencies Most Grateful Humble Servant, Lucy Paradise Be pleased to send your Letters to Mr. Anderson. P.S. I have hear that our good and great President is very Ill. God preserve his Life, but should any Accident happen to him I will Mourn as deep as for My Father, which is Six Months in Bombasin, Three Months in Black Silk and love Ribbons and the other Three in Black Silk and white Ribbons. Bishop Maddison brings this Letter.
Sir New York Novr. 25th. 1790 I feel at this moment, that it requires a share of Courage, thus, to solicit your attention, which even the liberality and benevolence of your disposition might fail to inspire; did not the anxieties of a mother surmount feelings, that in other circumstances would have prevented this intrusion; and impel me to address you on a Matter, that lays heavy on my Mind. My Eldest Son (who arrived here soon after your departure) has conceived an insurmountable Passion for the Sea. The bias originated in childhood and has been rivetted by his Voyage to Ireland. I find that his Studies have been directed to those branches of the Mathematicks and Natural Philosophy, connected with Scientific Navigation, Naval Architecture and Tactics, with an Avidity and perseverance, that mark, the force of his predeliction for the Naval line. He has also, availed himself of the opportunities afforded him in his Voyages to make himself a practical (as well as a Theoreatical) Seaman. I am told by Gentlemen of that profession, that his endeavours to attain the Art of Seamanship have been but too successful.—The fact is, that possessed of that decisive and collected Spirit, which rests upon itself; and of considerable Mathematical talents, he feels that Nature has furnished him with materials to gratify his ruling Passion to distinguish himself as a Sea Officer and Scientific Navigator; and is determined to pursue his Object at all Risks, and to spare no pains nor study that can ensure him success. Devoted to America, his Object is to serve in her Navy; which, with the enthusiasm of Hope, and the ardour of youthful expectation, he already pictures, riding triumphant on the Ocean, and giving Laws to Europe. Fully persuaded that a very few Years, will realize his vision, and America become a Maritime Power; he is desirous in the interim, of being employed in the only naval line in the Service of the Government which now offers; and wishes to get appointed Second in Command, to one of the Revenue Cutters; which, he understands are speedily to be put in Commission. The nearer the Station to the Seat of Government the more (he thinks) it would promote his Ultimate Views, in affording him opportunities of being known to the President and other distinguished Public Characters, and of obtaining a knowledge of the Politicks of America. I confess to you Sir, that my Son’s election of a Sea life is adverse to the Views I had formed for him, and the happiness I had  promised myself in his Society—but he tells me, that his happiness depends on being permitted to follow his bias for the Sea; and I have yielded mine to that consideration. He tells me too, that should he fail in his first wish, to be employed in the Service of his Country, he has determined to try his fortune in some Foreign Navy: an Alternative I anxiously wish to avoid. I have trespass’d Sir, on your leisure with this detail to request, that should an opportunity present itself, you would have the goodness, to mention my Son to the President as a Youth of some Talents, who has resolved to dedicate himself to the Naval line, and is desirous (for the present) to serve in the Station already mentioned, on board one of Revenue Vessels. Your good Offices, on this occasion, may make him happy in the attainment of his present Object, and prevent an Alternative, in which I contemplate the extinction of all my hopes in him. To any one but yourself Sir, the liberty I have taken would require much Apology.—Amongst the Characters who have access to the President there are some to whom I am better known, than I have the honor to be to you Sir, but there are none to whom my Mind would be more at ease, under a Sense of obligation. I have the honor to be with much Regard Sir, Your most obedt. and humble Servt., Henrietta M. Colden
January 23 1791 Permit me to congratulate you my Dr. Sir on Mrs. Randolphs safe delivery of a very fine Daughter and I with candor assure you she is uncommonly well with the sweet little Girl.—Mr. Randolph is not at home but every attention shall be payd to my Dr. friend Mrs. Randolph that is in my power and you will hear from us frequently.—I am with every Sentiment of Esteem and Affection your sincear Friend M Lewis
Marg[au]x en medocq Le 30e [de Janvier] de Lan. 1791 J’ay Recu, Monsieur, La Lettre que vous m’avés fait L’honneur de M’ecrire dattée du 6e 7bre dernier par Laquelle vous me demandés dix douzaines de Bouteilles de mon vin de Margaux première Calitée. Quoy que je Sois dans L’usage de Le vendre en nouveau, je m’en trouve une petitte partie de 3 tonneaux de L’année 1785 qui, quoy que bon, n’est pas une des meilleures Années. Il est tiré en bouteilles depuis plus d’un an, il doit donc avoir acquis à present le Bouquet qu’il peut avoir pour l’année. Je vous en envoye 4 Caisses de 25 Bouteilles chacunne à raison de 50 S. la bouteille, elles sont en verre de france, et une 5e Caisse de 25 Bouteilles de 1786 en verre Englais, à raison de 3 la bouteille. Je n’ay gardé q’un tonneaux de Cette Qualité, je l’ay fait tirer tout Exprès pour vous. Cette Espèce de vin a besoin de Séjourner quelques tems en Bouteilles, Comme vous Scavés, Monsieur. Apprès le tems que vous Croirés nécessaire pour en juger, vous pourrois me mander Si vous en voulés d’autre. Je pourrois aussi vous faire part d’une Barrique que j’ay gardé de L’année 1788, Et d’une autre Barrique de L’année 1790. Seulement, pour que vous puissiés juger Ces differentes années, Lorsqu’il Sera en Boitte, Si vous le jugés à propos, Monsieur, je me ferois un vray plaisir de Conserver toutes les années une partie de ma recolte pour entretenir la Correspondance que vous voulés bien etablir avec moy. Allors je Serais plus Surveillante que jamais pour qu’il Soit bien Conservé. En tout Cas vous pouvés Compter, Monsieur, qu’il n’y a aucunne fraude ny Mélange dans mon vin, Ce qui fait qu’il est très Sain pour la Santé. J’envoye, Celon votre intantion, les 5 Caisses de vin chés Mr. Fenwick, Consul des Etats unis de La Nouvelle Engleterre, aux Chartrons. Puisque vous le voulés Comme Cela, Monsieur, je Luy ferois passer avec ma lettre pour vous ma quittance de 325 pour le montant des 5 Caisses de vin, qui sont arrangées aussi Solidement qu’il a Eté possible. J’ay L’honneur dêtre avec La Consideration la plus parfaite, Monsieur, votre très humble et très obeissante Servante, Briet de Rausan Toutes Les dittes Bouteilles Sont Cacheptées de mon Cachet pareil à Celuy de ma Lettre. Mon adresse est: Mde. Briet de Rausan, Locataire au Couvent de Notre dame, rue Dutra, à Bordeaux. J’ay Monsieur environ 2 ou 3 mille Bouteilles de Ce vin de 1785 de 50 S. la bouteille, il est tout en Caisse de 25 Bouteilles, par Conséquent tout pret au Cas que vous en ayés besoin. Pour Celuy de 1786 à 3 la bouteille, je n’en ay plus que 3 Barriques.
Sir New York Febry. 5th. 1791 I am honor’d with your letter of the 20th. Janry.—If to know your Goodness be to deserve it, I might have some claim to the friendship you have extended to me; but the Benevolence of your Heart is the passport to that attention you have bestowed on my request and situation. In early life, and whilst my Sons were yet in Infancy, the Task to effect their Education, and to extricate their Property from embarrassments, devolved on me. In my endeavours to discharge this Duty, I was buoyed up by a Hope that their happiness and Success in Life, would Smooth the decline of mine, and retribute those days of Care to which my Youth was condemned.—Think then Sir, how interested I am to secure this Object? and think with what Satisfaction I perceive that my Son feels an additional incitement to distinguish himself, in being honor’d with your good opinion and Patronage. If he is so fortunate, as to do his Country service in the line he has chosen, the anxious moments his Fate may cost me, will be repaid. I assure you Sir, I received with unfeigned pleasure the News of the late Pacification. Had I been certain we should have continued Neutral, to have been a Spectator of the Miseries attendant on War, is sufficient to cause a feeling mind to sympathise, with fellow Creatures involved in that Calamity. To have participate in them, is to be impress’d with Horror at the Approach of that Scourge to the human Race. I am sorry the wants of this City have been so exaggerated as to excite a concern in our Friends; the price of Fuel has been uncommonly high, owing to the premature freezing of the Hudson; but there has been no Want of that necessary Article. From the same cause, we have been supplied with Wheat from the Southern States, but the plentiful Crops throughout the Continent renders the additional freight little felt. Our Stores up the Country are unusually full; when our navigation opens, this market will be overstock’d with Grain: an unpleasing prospect to Us Farmers. The attention of the Good folks of this City was lately engrossed  by the Choice of a new Senator to Congress, to the great Mortification of Mr. Schuyler’s friends. The Gentleman brought in by Governor Clinton’s party, as they say, “Not to Oppose, but to keep a Sharp look out on the Measures of the Government” is a Man of too considerable abilities, for the Side he has taken. If he moves on Antifederal Ground, he may do harm. But that business is now over. Our present Object is The National Bank: in confidence of that measures being adopted, large Subscriptions have been agreed upon amongst our Monied Men and Speculators; and Committee is sent to Boston to unite their Measures with the Stockholders in that City. I am credibly informed that Subscriptions to the amount of nearly two Millions of Dollars will be fill’d up in this City and to the Eastward. The confidence and unanimity amongst the Publick Creditors, in N. York and Boston will give them a powerful influence in the direction of the National Bank; to effect this, every Nerve will be strained by the Monied Men and Mercantile interest in these Cities. My Son requests me to offer his grateful acknowledgements to you Sir. He is very impatient to have the honor to be personally known to you, and to your Friend Mr. Rittenhouse. Having devoted himself this winter to the Study of Astronomy, he contemplates with pleasure the advantages to be derived from Mr. Rittenhouse, in that Science.—I have the honor to be Sir, With perfect esteem, Yours Most Obedient & Obliged Servant. Henrietta Maria Colden.
Dear Papa February 13 I am very sorry that my not having wrote to you before made you doubt of my affection towards you and hope that after having read my last letter you were not so displeased as at first. In my last I said that my sister was very well but she was not. She had been very sick all day without my knowing any thing of it as I stayed upstairs the whole day. However she is very well now and the little one also. She is very pretty has beautiful deep blue eyes and is a very fine child. Adieu my Dear Papa & beleive me to be your affectionate daughter, Mary Jefferson
Dear Sir March ye. 1st. 1791 London I hope you will pardon My troubling your Excellency with these few lines to make an enquiry after your health, and that of your amiable families: And to know at the same time if you have received a Letter I did Myself the honour to send you by Bishop Maddison. You will greatly oblige Mr. Paradise and Myself if you would let Us know by the return of the Spring Ships whether your Agent has remitted you the Money you were so good as to advance Mr. Paradise during his stay at Paris, as our Merchant Mr. Anderson has paid it to your Agent. Mr. Paradise joines with me in respectful Compliments to You, and your good family. Dear Sir I have the honour to be Your ever Obliged Friend, Lucy Paradise Please to direct your Letter to Mr. Anderson No. 10 Crosbey Squr. Bishopgate St.
march 6 According to my dear Papa’s request I now set down to write. We were very uneasy for not having had a letter from you since six weeks till yesterday I received yours which I now answer. The marble Pedestal and a dressing table are come. Jenny is gone down with Mrs. Fleming who came here to see sister while she was sick. I suppose you have not received the letter in which Mr. Randolph desires you to name the child. We hope you will come to see us this summer therefore you must not disapoint us and I expect you want to see my little neice as much as you do any of us. We are all well and hope you are so too. Adieu dear Papa I am your affectionate daughter Mary Jefferson P.S. My sister says I must tell you the child grows very fast.
[Monticello] March the 26 It is three weeks my Dear Papa since I have had a letter from you. However as it is now my turn I shall not be ceremonious. We are all waiting with great impatience to know the name of the child. Mrs. Lewis was so kind as to give me a Calico habit. Adieu my Dear Papa. I am your effectionate daughter, Mary Jefferson
Dear Brother Chesnut Grove april 6 1791 I receiv’d the favour of yours dateed october wherin I found a total disappointment of the happiness I had long flattered my self with of seeing you, it being at a time that our distress cannot be  describ’d. It is too much for my pen so that I will not trouble you with it. You must now permit me to hail you grandfather and I do Sincearly congratulate you on the happy occation of pat’cyes safe recovry. Wee anticipate the pleashure of seeing you this spring as your anxiety must be very great to see the little Stranger. I am sorry to tell you our sister Carrs ill health becomes seriously alarming. She has an obstanate languid fever that does not intermit for two or three days. When they leave her she is considerable weakened and they return very frequently. I very much fear these are dropsical simtoms. I have long wisht an oppertunity of answering yours. Had it not have been for Mr. Epps’s politeness to me it was uncertain when I should have been gratified. Mr. Bolling Joins in love to you and apologyes for his negligence. The rest of my family Join in love. Adieu my dear brother may every blessing this life affords attend you are the most ardent wishes of your affectionate Sister Mary Bolling
Dear Sir Eppington, April 6. 1791. I sincerely congratulate you on the birth of your fine Granddaughter. I am told she is a nun such. I wish’d much to have been with my dear Patsy, but it was impossible. I now please myself with the happiness of seeing them all soon. The number of advantages you would have for my dear Polly in Philadelphia I have no doubtmust be very great, added to the happiness of your having her with you, tho’ be assured my dear Sir it will always add to our happiness to have her with us. I never met a more governable temper. Your kind intentions with regard to Wayles have delight’d me not a little, as I am **sure** if he follows your directions (which I hope there is no doubt of) I shall be completely happy. We all join in wishing you every blessing this life affords, Your affectionate friend, E Eppes
Plymouth, Massachusetts, 10 Apr. 1791. Transmitting a volume of her poems to be registered for copyright.
Dear Papa Monticello, April 18th, 1791. I received your letter of March 31st the 14th of this month; as for that of March 9, I received it some time last month, but I do not remember the day. I have finished Don Quixote, and as I have not Desoles yet, I shall read Lazarillo de Tormes. The garden is backward, the inclosure having but lately been finished. I wish you would be so kind as to send me seven yards of cloth like the piece I send you. Adieu, my dear papa. I am your affectionate daughter, Maria Jefferson
Dear Papa Monticello May 1 As Bob is going down the country to morrow we shall all write to you by this opportunity. We expect jenny and nancy Randolph here in july. Mr. Randolph has bought a horse called my heart and a saddle for me to ride out on also a pretty whip. My niece is prettier and prettier everyday. This place is beautiful now. The peaches cherrys and strawberries are very big allready and there are a great number. Adieu my dear Papa I am your affectionate daughter Mary Jefferson
My Dear Papa [Monticello] May 29 I am much obliged to you for the veil that you sent me and shall allways were it. I have began to learn botany and arithmetic with  Mr. Randolph. The mare that he bought for me is come. She is very pretty and is sister to brimmer. She can only trot and canter. The fruit was not killed as you thought. We have a great abundance of it here. Adieu Dear Papa I am your affectionate daughter, Mary Jefferson P.S. Little anna grows fast an is very pretty
My Dear Friend Paris june the 7th 1791 Altho Every Motive of Regard and Affection Conspire to Make me Lament our separation, I Lament it still more on Account of our Revolution wherein Your Advices would Have Greatly Helped  us, and Could Not fail to Have Had a Great Weight Among our Constitution Makers. You left us on the Point of that dissolution of the Patriotic Party which I then did, and I Have ever in Vain Endeavoured to Unite into Mutual Concessions of self interest, and self importance. You left me on the Eve to Add Greatly to My Responsability with the Nation, as the Assembly and the King Had Not Yet fixed their Abode Within these Quarters. So Many Courts Have passed Since, So Many Accounts Have from time to time Been sent to You that I will only say the Assembly is Shortly to Make Room for all Constituted Powers. How far we May induce them to Mend the Constitution Before they Go, I do not know. This I Can say that it will not Be a Voluminous Work, and that leaving Much to Be Corrected By Common legislation, We May Hope this Constitution, such as it will Be Presented, will Have a fair trial, and Give time to prepare a More proper one for the Next Convention, to Be Convocated in a short period. At least our principles are Sound—Liberté, Egalité, are the Motto, and if the people Can Be Brought to a proper scale of obedience to the Law, which much depends on some Alterations to Be Made in the Administrative and judiciary parts of the Constitution, We May do Very well, and Apologize for the work in Considering By whom, and for whom it Has Been Made. May Education Better fit for liberty the Ensuing Generations! Enough it will Be for this, to Have Broke the fetters, destroied Prejudices, and laid a Good Ground to Be Sowed By our successors, and Cultivated By posterity. M. de Ternant will Acquaint You with the state of our internal and External Politics. To Him I Refer Myself. You Have thought with me He is the Best Man to Be sent to America. I Had in the Course of the Revolution Many opportunities to Experience His Abilities, Honesty, and friendship. He will deliver to You a letter of our National Assembly, Expressing sentiments Most sincerely felt. He will Explain How it Happened the Assembly Blunderd in the duties on oil and tobacco, and I Hope You will Be satisfied with His Accounts on these Matters. Our friend Short who Continues to Conciliate to Himself Universal Esteem and Affection will write to You Respecting New orleans. France will do what she Can with Spain, and that is Not a great Guarantee for the Negociation. I Had My Private sentiments long Ago fixed on the subject, and Never thought it was possible, Much less desireable to prevent Louisiana from declaring Herself independant—That You Could Not stop an Adress from Kentucky to Congratulate them on the Occasion—and that it should Be Hard  Not to Have Provided a Bundle of Proofs that Spain Has Been in twenty Circumstances the Agressing power. Adieu, My dear friend, Remember me Most Affectionately and Respectfully to the two ladies. Most Friendly I am for ever Yours, Lafayette
My dear Papa Monticello, July 16th. 1791 I have received both your letters, that from Lake George and of June 26th. I am very much obliged to you for them, and think that the bark you wrote on prettier than paper. Mrs. Monroe and Aunt Bolling are here. My aunt would have written to you, but she was unwell. She intends to go to the North Garden. Mr. Monroe is gone to Williamsburg to stay two or three weeks, and has left his lady here. She is a charming woman. My sweet Anne grows prettier every day. I thank you for the pictures and nankeen that you sent me, which I think very pretty. Adieu, dear papa. I am your affectionate daughter, Maria Jefferson
Dear Sir Frederickbg Academy November the 3d. 1791 I take the liberty once more, to beg your friendship for my eldest Son Walker Randolph Carter, who is now on his way in the Packet for Philadelphia, to live with Mr. Hunter a Coachmaker, who takes him for two years without fee. I am in hopes I shall be able to cloath him decently and allow him a little pocket money. All I have to ask of you is, that you will be so good to give him your advice and countenance. He has a tolerable education, but fear he may loose what he has learned, unless some friendly person, will sometimes  put him in mind of the bad consequences that will attend his not reading all leasure hours. Any Books you will think propper to recommend and lend, I make no doubt he will be thankfull for, as I can asure you, as I told you in a former letter, that his disposion is very placid, but rather to diffidently backward to go through this life with out the encouragement of a friend, as such I flatter myself you will be. The Atorney has procured the place for him and make no doubt will act a friendly part, but a parents tender feelings woud have all the world look with their eyes on their Children, which must plead my excuse, for now soliciting your attention to my Child, who is in a strange place, at so great a distance from his parents. Mr. Carter has written to several of our Country Gentlemen on this subject, who I hope will all have an eye to the conduct of our Son, and advise him whenever they see Occation. Our Second Son Charles Landon we have placed with a Phisician in this Town, to get acquented with medecine, after which, if our finances will allow, we shall send him to Edenburg. I shoud have preferd Philadelphia, coud we have fixed him there without paying so enormous an admistion fee. After beging your forgivness for this intrution will subscribe myself your obedient & much oblig’d, Eliza Carter
Dear Sir Wednesday London Febry. the 1st. 1792 We have received your Excellencies very kind and friendly Letter, and return you our grateful thanks for the Interest you are so good as to take in our affairs, we beg you only to Continue your good offices, as we stand in greater need of them now, More than ever. We thank you for the advice you give Us concerning the Thousand Pound Sterling we have in the Funds of Virginia, and shall certainly follow your Excellencies plan. Our worthy and Constant friend Dr. Bancroft agrees  with your Excellency: And for which purpose the Dr. and the trustees for the Creditors, have written to My Cousin Colo. Nath. Burwell, and our Steward, to Authorise them to Sell out the Money and remit it to them, as soon as possible. But to these important Letters, which were sent last year the Trustees have received no Answer, which Makes them extremely Angry and Mr. Paradise very unhappy. I am certain if our Steward would do his duty properly, and send us good Bills of Exchange upon respectable Merchants here that no time should be lost in protesting them, what with the Thousand Pound I have mentioned, and Norton’s Debt, and the Annual Income of the Estate, we Shall Certainly be cleared of all the Debts this year. Our Steward ought to exert himself in the best, and most attentive manner he is able, as the Great Sum of Money we loose in the Funds of Virginia is owing to his Father or himself accepting of the Money during the war, which he had not our authority for so doing. The whole of My Estate when we were in America was in a very bad Condition, owing intirely to the attention our Steward gave to his own affairs and none to ours. Thus my Most respectable and kind friend I must intreat you to write a private and pressing Letter to My Cousin Burwell and desire him to let you know the following particulars. What quantity of Stock of all kinds we have upon each Quarter as My Estate is divided into Six parts. Arches Hope, Cheapokes, College Quarter, Rich-Neck, Drinking-Swamp, Deep Spring, and Three Houses in Williamsburg. I wish to know the Number of Houses, Negroes, the Names of the Tennants, the Number of Acres, the Number of all the Cattle, and the Number of the Overseers, and the Number of the Carts, and Waggons &c. at Each Quarter; and also what the Steward is yearly paid for the Care of the Estate, and the wages of the each Overseer and what the Ammount of the taxes that are yearly paid; for the Estate, and what the different things are, that pay taxes, and as near as possible what the yearly Income may be. We should be obliged to you to advise Colo. Burwel not to send an[y] more tobacco to this Country as all the profites go to the Merchant and the King. By Complying with My desire, we shall know the true State of our affairs in Virginia, and be able to regulate them properly, that in case I should die, I may have put them, in such good order, through your means, that Mr. P. and My daughter shall have no trouble in the Management of their affairs. I fear, I have taken a very great liberty with you, but knowing the sincerity of your Excellencies Heart, for Mr. P., and Myself to be put into an Easy, and respectable Situation, I have taken the liberty to beg of you to write a private Letter to Colo. Burwell, that our Steward may not suspect that it is for us, to get  after all our Misfortunes a Comfortable Subsistance, such as we have a right to. I Beg Your Exelencies kind exertions in what I have desired; you will make Me beloved by My Husband, and respected by our friends. Mr. P., is never drunk but is at present indisposed. He loves, honours and respects you, and feels with gratitude the attentions you are so good as to honour Us with. We join in affect, and Love to you, and your amiable family. I beg leave to acquaint you that the Peace between the Russians and the Turks was concluded on the 29 of Decr. 1791 old Style which is the 9th. of Janry. New, with a full grant of all that the Russians demandes both from the Turks, and the English. Adieu our protector and best of Friends, and may every action of yours be Crowned with Success. I have the Honour to be your Exellencies Grateful Humble Servt., Lucy Paradise Please to direct your Letters for Us at No. 87 Titchfield Street. Queen Ann St. East
A Paris ce 13 Mars 1792 Un nouveau motif Monsieur m’éngage à vous présenter mes actions de graces pour touttes les bontés que vous avés pour mon Cousin. Vous avés pris la peine de lui donner le Certificat dont il avoit besoin pour toucher son Legs. Assurément jamais certificat ne valut davantage, ayant votre signature et le sceau de l’Etat. Je vois dans ses Lettres que vous lui témoignés en bien des manieres une amitié tres précieuse pour lui. Le moment de votre arrivée à Monticello et le séjour que vous y faites lui procure mille agrémens auxquels il est tres sensible. J’en ay  les détails et ce n’est que comme cela que j’ay le plaisir d’apprendre de vos nouvelles. C’est lui qui me rappelle à votre souvenir, sans cela je crois Monsieur que vous m’auriés tout à fait oubliée en oubliant le françois, que vous dites ne savoir plus. Pour moi je saisis cette occasion pour vous assurer que je ne vous ay point oublié, que je regrette bien d’avoir étté si tôt privée du plaisir de jouir de votre societté. Le sujet de nos conversations est devenu intarissable, j’aurois eu bien besoin de l’adoucissement que vous saviés mettre à mille maux naissans desquels vous aviés une idée tres différente. Je veux avant tout Monsieur vous parler des difficultés que mon Cousin éprouve à recevoir son Legs. La Procuration qu’il m’a envoyée dernierement pour la faire passer à son Oncle est rigoureusement insuffisante, elle n’est point expresse pour toucher le Legs. Il faudra que l’oncle veuille bien être caution vis à vis des Exécuteurs Testamentaires. Une autre difficulté plus facheuse encore c’est celle pour lui faire parvenir la somme dont les deux tiers seulement sont échus. Mon Cousin ignore aparemment que par la perte du Change qui est de plus de moitié il n’auroit à peine que 5 mille francs pour dix; il me mande que surtout on n’employe pas la voye du Commerce parce que ce seroit trop longtems à lui parvenir. J’ai envoyé à l’oncle chargé de la Procuration l’endroit de la lettre. Il pense comme moi qu’il est hors de raison de lui envoyer autrement que par la voye du Commerce. Je lui ay envoyé à Nantes tous les renseignemens que j’ay eu de bonne part d’un Négociant appellé Humbert qui demeure à Paris et qui a une Maison de Comerce à Boston et une très riche au Havre. Il prendra touttes les informations nécessaires avant de s’y addresses. Mr. Humbert connoît tres bien ce qu’il faut envoyer à Boston et à l’arrivée du Bâtiment celui qui a mis des fonds doit toucher les deux tiers de la valeur du chargement en Piastres et le reste après la vente. J’ay l’honneur de vous marquer tout cela Monsieur pensant que s’il y a des moyens que vous connoissiés pour lui procurer ses piastres promptement en Virginie vous les lui indiquiés. C’est toujours comptant sur vos bontés pour lui. Vous connoissés deux causes différentes qu’il a d’être pressé de toucher. Monsieur Shoort conseilloit d’attendre à faire passer, que l’argent nous fut revenu à la place des assignats. Je ne sais comment à travers tous les événemens à quoy cela tient il peut fixer un point à quoi s’arrêter. Il va donc nous quitter pour aller en Hollande. Il regrettera dans ce païs ci ses chers économistes. Il verra dans les autres païs le cas qu’on fait d’eux. Monsr. Moris vient occuper la place que vous aviés Monsieur. J’ay déja prié Mr. Shoort de me faire faire connoissance avec lui. Peut-être voudra til bien me rendre le service de continüer de me faire parvenir plus surement et plus promptement mes lettres. Pour revenir à cette liberté à cette  philosophie qui devoit nous rendre heureux, elles sont si avilies si humiliées en France par ceux qui égarent le peuple qu’il y aura à leur faire le reproche de nous avoir privés peut-etre pour toujours des avantages qu’on s’en étoit promis. Cette liberié n’est autre que la faculté de pouvoir impunément résister aux loix, il n’y a aucune force pour faire obéir. La loy de l’impôt est impraticable quant aux terres, aussi ne se paye til point. Le pouvoir Judiciaire et surtout le code Criminel est fait, pour favoriser tous les Crimes qui se multiplient d’une maniere effrayante. Il a eu déja autant de sang répandu que dans une Guerre, à la différence près que c’est par des assassinats prémédittés, exécutés avec une cruauté inouïe, et que la perte des biens s’en est presque toujours ensuivie. La Banqueroutte nous arrive. Et pour parler des nouvelles actuelles, on vient avant hier de recevoir la nouvelle de la mort de l’Empereur, on esperoit qu’il remedieroit de maniere ou d’autre à nos maux. Ce prince si pacifique et si sage est mor le 3e. jour d’une maladie dans laquelle il avoit des vomissemens. On l’a saigné, dit on, mal à propos. Et c’est la le moment ou l’assemblée traduit à la Haute Cour de Justice le Ministre des affaires étrangères pour une réponse faitte à Monsr. de Kaunits, laquelle lüe à l’Assemblée fut tres applaudie. Trois autres ministres ont donné ou reçu leur démission. Enfin on renverse de fait le Thrône. Dieu veuille conserver les jours du Roy. Je finis Monsieur par vous demander pour Mon Cousin la continuation des sentimens que vous lui avés montré jusqu’à présent. Il seroit bien malheureux de cesser de les mériter. Dans votre absence il lui reste encore l’avantage de faire sa Cour à Madame Randolphe, il me paroit qu’il profitte souvent de son voisinage et de celui de Monsieur Randolphe. Je desire qu’il recoive son Legs à tems pour faire l’acquisition que vous lui conseillés et qui, le laissant près de vous le fera jouir de tout ce que j’ay perdu. Soyés persuadé que j’ay aprécié autant que personne le prix de votre societté et que les sentimens d’estime et d’attachement qu’elle m’a inspirés ne finiront qu’avec moi. J’ay l’honneur d’être Monsieur Votre tres humble et obeisste servte. Plumard Bellanger
Paris, 22 June 1792. She has already thanked TJ for the certificate he gave Derieux concerning a bequest of 15,000 francs from his uncle and now asks him to use his good offices with Robert Morris to facilitate the prompt and advantageous sale for Derieux of a cargo of merchandise worth the 10,000  francs due on the bequest, which Derieux will receive in this form rather than cash because of the unfavorable exchange rate. She complains that TJ does not write frequently and makes her jealous by treating Derieux so well. Impressed by the fine laws of the United States, she regrets that there is no counterpart in France to the veto power the President has recently exercised. M. Darcel, who made up the cargo for Derieux and whose bill is enclosed in summary form, was in Philadelphia during the American war and is known to TJ. She has not yet been able to meet Gouverneur Morris, but he has graciously agreed to forward her letters. Despite political differences with Short, she regrets his departure and has told him that in the country he is going to, or in any other country, he would see laws destructive of every principle, whether religious or financial, that have been given by his friends “les Economistes,” whom he still admires. They agree that Gouverneur Morris does not admire the laws of France or their consequences.
Sir Boston July 15th 1792 The letter which your honor has pleased to favour my husband John de Neufville with did not come to hand till the 12th of this month, Should not have failed to answerd it imediatly had he received it Sooner; Mr. John de Neufvilles bodyly indispossition prevents him the honor of writing him Self to you, you will therefore be pleased to excuse a femalle advocate, unacquainted with your Language, but anxious to comply with your demands, to State these facts, to which  Mr. John de Neufville will lay his oath of, if you think it necessary; his losses are in the first place goods which he has Send to St. Eustatia to a Mr. Hawkins, to facilitate, or Supply, either congress or individualls, with what they may have wanted at that time; which where taken and confiscated at the time the Iland was, as goods belonging to a Rebell, he was called Such on account his having Planed and Sined the Commercial treaty Betwen the U,S, and holland, the losses of the goods were 6 thausen pounds Sterling; he further Supplied goods for South Carolina, which went in those frigates, Commanded by Commodore Gillon, to the amount of 15000 pounds Sterling. The British politick checking Mr. de N. Credit on all Side his crediteurs pressing upon him to answer the demands for them goods, he found him self necessitated to Sell his Elegant Mansion house in Amsterdam, upon which he lost 4000 pounds Sterling; he might have evoided it, would he have Sacrifice the American Cause and Credit; he 3ly found it necessary to promote the american cause and credit, to reduce if Possible the Credit of the British Stock; he therefore Sold his Shares in them and Sustained the lost of 5000 pounds Sterling; he 4thly Kept an open tabell for 3 years his house was then the only house for the american cause to result to, as well Public or Private characters; his hospitallities and travell during that period, which are to innumerable to Specifie, has cost him at least 10000 pounds Sterling; Major Jackson was at that time in [Holland?] and was an Eye witness of Mr. de N. hospitalties, and travells to Paris for pollitick or Commercial casses; I Supose he will not refuse you a Testimony there of, as every Patriotick Soul must feel for one an a n’other; it would be difficult to Prove immediatly the losses above mentioned, as the Persons and papers are in Europe where it has happened, and Mr. de Neufvilles Sons miend not being altogether Calculated to Keep his Books nor papers in order has been already the cause of innumerable losses; if you will be satisfied my husband declaring this Statement upon oath before the justice of peace, I shall send it to you as soon as you desire it, if you will favor me with a Line in which manner you wish it to be. As to the result of Mr. de Neufvilles exertions in promoting the Commercial treaty betwen the u, S, and Holland, I must refer you to william Lee Esqr. letter, which is in your Possession, and left as Mr. gerry informs me with the Memorial—Certain it is that the British have persued Mr. de N where ever they Could, and have Ruin him entirely on account of my husband being the first in holland whom took the cause of the Americans at heart, and may I even be bold enough to assure you that he was the only author in promoting to  effect that the Duth goverment became in favour of america—to Show you his unbounded attechemet to your Cause, notwithstanding the Risk of his life and fortune hung but on a Silken thread, when Henry Lawrence was captivated, and in the dunjon of Londen, and his own Country could not assist him at that moment, he gave him Credit of two thausen pounds, when Mr. Lawrence was relissed and at Liberty he came to holland to thank him for his Kindness—to prove further what Little Support Mr. de N had at that time in his own goverment, in favour of your Cause, a gentleman belonging to the Court of the Hague came with instrution from powerfull hands of that Court, to Mr. de N. and told him that if he would drop the american Cause, they would give him as much Credit as he should want in the commercial Line, and above that an Employ, which should bring him in yearly near five thausen pounds; and all the answerd Mr. de N gave him, was, that he woud Sacifice him Self, and that a free Nation would be dearer to him than to be made passesor of [Peru?]; I can peldge my oath for this conversation which I was witness too at Bonn, at the Court of the Elector of Cologne, Mr. Heathcot Minister Plenipotentiary from the British court at this, Shook hands with Mr. de Neufville, and told him Sir, it is fortunate for you, that we are at peace now with america, your life would not have been So Secure could I have obtain you 3. 4, years ago I had orders from my goverment to persue your life and fortune where ever I could. When we came to america in 1785 the demands of the house of John de Neufvill & Son, were nearly Sixty thausen pounds Sterling, which consisted in Congress, State, an Private debts; of which Mr. de N. has Little nor any thing recovered; his great humanity and Lenity of dispossion in not pursuing the Private Debtors on account of theire pleading losses Sustained during the war, and Law at that Time was not in Some of the States in favor of recovering very expedisiouly any demands, and as Private debts Seldom growes better by delay, has Ruin Mr. de Neufville So far, as that he is now in his infirm State of body beholden, to a few of his Relations from holland, for his dayly Subsistence in a Country boarding house, which can be atested by Mr. Stephen Higginson of Boston, Mr. Cazenove a gentleman from holland, Mr. James Searle a gentlemn of Philadelphia, can testify like wise Mr. de Neufvilles excertion in the Cause of America—Shall write to Henry Lawrence Esqr. by the first opportunity, and as amost expedisious way of information have taken the Liberty to inclose a line to Major Jackson, for which I must claim your honors Candour; if true Patriotism and Personnell Ruin merits rewards Mr. de Neufville  will undoubtely find it by your excertions, in a Republican goverment, whose Basis is Philantropi—I have the honnor to be with a profound respect your honnors very humble Servant A, M, de Neufville
Paris, 14 Sep. 1792. She should cease importuning him with her letters in French, but his rare kindness towards Derieux compels her very humble thanks. If Derieux, who has used TJ’s credit, could not pay, she would certainly do so, but she believes he is not so badly off that he cannot honor his business engagements. She reproaches TJ affectionately for his wish to forget French. [P.S.] She is **sure** he can master the reading of this letter.
Monsieur [ca. 6 Oct. 1792] Je vous Supplie pardonner deccuser madresser a vous pour vous prillier de medonner des nouvelles de Mr. Hallet mon mary. Voisi quel que ten que je nen neresue de lettres et jensui inquiete. Deplus je vous demende la permisons de vous decrire laposisions dans laquel jesui depuis Son absenses. Le 5 ou les 6e aoux il partie dappres les hordres quil enresue de Mrs. les comisere. Je reste moi et 3 enfens a vec tres peut darjans que peu melese mon mariy. Depuis Son absense, je ne sesset detres malade. Unedes Suite hordinere dela posisons dans laqueljesuis car veullie apprendre que jesui Sur lepoin dacouche, Sequi me met tres inquiete. Deplus leproprietere dela maisons que jocuppe metourmente pour lui peillier monterme échu le dernier a oux. Non Seullemen jenelepui peiller mais jenepui moi meme meprocurer les chose depremier nesessitee. Je menque de tout il fait unfroy horible a lacampajne mes enfens enson incomode et jenepui les Secourire. Veullie mepermettres Monsieur dinvocquer isi votre clemens envous prillien de macordere unleje Secoure Sur les honorer que vous  Soupsonnes pourons aitres acorde a mon mary. Permettez moi encoredevous demende Si vous croylie quil poura seficcez a jorjetonne dans Sequa je voudrois pouvoir li rejoindres pendens que jelepuis encore car dans lecouren dumois de novenbres jeneseret pas trensportables. Degne repondres atoute mes question et ne les pas trete dehardiese. Deigne jeter les eux Sur laposisons ou je metouve et Sur Selle ou je mesui trouve enfrense. Jesui fillie de feux Me. Gomain Greffe au parlemens deparis alie de Mr. Rolland presidans. Jeresue lameme educcasion que Mde. delafaliete avec laquel julonneur de reste 3 anne aucouven apres enaitres Sortie elle monora deson amitie et Sefut meme elle qui fut enpartie lacose denotre avenueicy. Je di denotre jai tore car elle et lemarqui de lafaliete medie derestere que javois unappartemens dans leur hotelle que mes enfen reseverois les mesoi que les leur enfens deleur presepteur et quil ferois ensorte que jemapersuse le moin posibles delabsense demon mary quil preparerois les lieux et que lorquil Serois enetat de meresevoire il mapelerois aupres delui, que lenploi quil alois avoire dans lacompagni du cioto nous rendrois Sou peut 2 mil cent derente, mais que Comme il faloi forme des ville &c. que jeprouverois des desagremens et quil les falois eviter. Je remercie endisen que lhonneur que lon moffroi meferois le plus grand plesire Si jelepartajois avec mon mary querien aumonde nemedeterminerois a lequite, quepartout jelesuiverois Seque jai fai. Je nemerepens pas et jedie meme que toute lapesn et lamiser quejai eprouve dans les peillie des Sovaje nesrien pour moi puis que jai conservez mes enfens et leur pere. A pardonnez moi mafebles Si Sen est une, en mepermetent devous dire que jamais nefut plus digne des Sentimens quajamais jeprouvere pour lui. Il est bon [quom?] bon mary et a toute les qualite quil faut pour Se merite lestime et les bonte detoute les personne qui veul bien prendres la pene de lejuge. Je memet a vos pie pour Sollisiterpour lui la continuasions de votreprotections. Jai lhonneur Detres avec respect Monsieur Votre tres humbles et tres obeisente Servente Gomain Hallet
Sir Treasury (Virginia) 10th. Decemr. 1792. Last Evening I received the letter you honored me with, dated the 13th. Ulto., and this Morning I have paid ten thousand Dollars, in part of the second instalment of the Monies voted by our Assembly towards the public Buildings at the Seat of Government of the United States: the residue will be paid, as soon as the state of our Treasury will enable me. I am, with very great esteem & respect, Sir, Your most obt. Servt J: Ambler
My Dear Brother Spring Forest. Decr. 15. 92 You wrote me some time ago that you could discharge any pressing debts for me. My haveing a Daughter Married and to fix off for Kentucky after which it is hardly probable that I shall ever see her agane has put me to some unusual expence this together with my bad management Obliges me to ask your assistance. If you could let me have Seventy five or Eighty Pounds you would do me a great favor. Mr. Myers the Merchant I have delt with has but a small Capatal and wishes to receive payment of my account before he purchases his spring goods. He intends to Philadelphia and offers to call on you if I will give him an Order. I hope my Dear Brother this will not put you to inconvinience but write and let me know perhaps Mr. Myers might be put off untill next fall. The pure air of Monticello has proved in my case so efficacious that I have not had a return of my fever since my return from thence. I am now recruiting but any distress or Anxiety of mind has always thrown me back and I at present look forward with dred to a piriod which approaches when I am to be separated (perhaps forever) from an Affectionate and dutyfull Child. Adieu Dear Brother [Yours Most] Affectionately M Carr PS Present my best love to Dear Polly MC
London February the 17th. 1793 You will say my dear Sir after the Long silence I have observed that a line from me has lost its effect. But remember that I plead guilty to the charge of Idleness only; for when my friends require my assistance few are more willing than myself and there is no occasion in which I take more pleasure than in warmly recommending to your Attention the Count de Noailles, one great object of his visit to America is to render service to his Brother the Marquis de La Fayette, whose fate America must see with sorrow and indignation. Catherine unites with me in best remembrances to yourself and daughters she hopes to see her young friend in a few years. Accept my dear Sir of my grateful remembrances for the pleasure you procured me at Paris, and the assurances of my friendship. Angelica Church
Sir The Retreat 9 May 1793 You have heard of the irreparable loss I and my family have Sustained. Your friendship to me formerly emboldens me to trouble you at this time. In a letter I lately received from Coln. Humphreys he advises me to make application to you, that the person who should be authorised to take possession of the public property that was in Mr. Barclays charge, may be directed to deliver his private effects to the order of the person or persons empowerd to receive them. John Barclay Esqr. of Philadelphia is the person to whom I would wish Mr. Barclays private effects to be sent, and your giving directions that they may be delivered to his order will add to the obligations, confer’d on me by your attentions to myself and family in France, and with them will be ever gratefully acknowledged by Sir your most obedt humble Servant Mary Barclay
Down Place august 19. 1793 Your letter my dear Sir arrives in time to encourage me to solicit your friendship for a friend of yours and mine, as well as to thank you for your attention to my recommendation. How changed are the fortunes and  situations of those we loved at Paris! and whose Welfare were dear to us; La Fayette is in prison at Magdebourg; and enclosed is the extract of a letter he has been so fortunate as to find means of conveying to a friend and relation, who has sought an Asylum in this country; His Love of Liberty has rendered him culpable in the eyes of a Despot: and you Sir cannot read the recital of his sufferings without tears. General Washington’s interference is the only hope left to him and his family; Madame de Corney is a widow with a very limitted fortune, and retired to Rouen; Mrs. Cosway gone into a convent at Genoa: Monsieur de Condorcet under accusation, but fortunately escaped or concealed in France, Custine a l’abbaye a sacrifice for the fall of Valenciennes. And the Queen of France at the conciergerie, and taking her Tryal, Marat assasinated by the Republicaine Cordét, who suffered Death with the fortitude and tranquillity of Innocence; I hope soon to remove from the vicinity of such horrors; and retire to America, where I shall look forward with impatience to bring Catherine to her friend Maria, and to renew the assurances of my friendship and to express the satisfaction I receive from Your friendly letter: Angelica Church
Lafayette to the Princesse d’Hénin Magdebourg ce 15 Mars. [1793] Imaginé vous une ouverture pratiqué dans le rempart de La Citadelle et entouré d’une haute et forte palissade; c’est par la qu’en ouvrant successivement quatre portes, dont chacun armée de chaines, Cadenats, et Bars de fers, on parvient non sans peine et sans bruit jusques a mon cachot, large de trois pieds, et long de cinq et demi; il est Lugubre, humide, et m’offre pour tout ornemens, deux vers françois qui riment par souffrir et Mourir. Le mur du coté du fossé, se moisée, et celui du devant laisse voir le jour, mais non pas le soleil, par une petite fenetre grillét; deux sentinelles dont la vüe plonge dans mon souterrain, mais en dehors de la palissade, pour quils ne parlent pas, des observateurs etrangeres a la guarde; et puis tous ce qu il y a de murs, de rempart, de fossés, et des gardes, en dehors, et dedans de la citadelle de Magdebourg. La Bruante ouverture de mes quatre portes se renouvelle le matin pour introduire mon domestique; a diner pour manger en presence du commandeur de le Citadelle et de celui de la garde, et le soir pour ramener mon domestique en prison, et apres avoir refermé sur moi toutes les clefs, le commandeur les emporte dans le logement ou depuis notre arrivée le Roi lui ordonné de coucher. J’ai des livres dont on ote les feüilles blanc, mais point des nouvelles, point des gazettes, point de communication, ni encre, ni plumes, ni papier, ni crayon, c’est par miracle que je possede cette feüille, et je vous ecrit avec un curedens.— Ma santé se deteriore journellement: ma constitution physique a presque autant de besoin de liberté, que ma constitution morale; le peu d’air qui m’arrive en sejournant dans le souterrain, detruise ma poitrine, la fievre sans mele souvent, point d’exercise, peu de sommeil; je ne me plains plus, et sais par experience qu’il est au moins inutile de le faire savoir; mais je m’obstine a vivre, et mes amis peuvent compter sur la reunion de tous les sentimens que ma porte a la conservation de moi meme, quoique d’apres ma situation, et le progress de mes souffrances, je ne puisse pas repondre longtems de leur efficacité, peutetre me vaut il mieux les affliger d’avance, que de les surprendre aussi triestement.
My Dear Sir Nov: 5th. 93. I have heard very melancholy accounts from Philadelphia occasioned by a fever and in consequence am extremely anxious for you and my young friend Polly: I hope that you have both escaped the power of its destructive influence. Accept of my good wishes, for your health, and be so good as to write me a line to say that your family have enjoyed security from this disease. Adieu A Church
Paris, 1 Feb. 1790. He asks TJ to be of service to M. Minguée, who is leaving for America to “faire un établissement” on the Scioto river, having served with patriotism and zeal in the national Parisian army from the beginning of the Revolution, and from whose great friend, the “Gouverneur de mon fils,” he has just received new testimony in his favor. [P.S.] “I don’t know, My dear friend, When this will reach You, Accept My Most afectionate Compliments, and Warm Wishes for Your Speedy Return.”
Dr Sir [ca. 30 May 1792] Mr. Madison consented that Mrs. Pine Shou’d finnish a portrait of him self—which was began by Mr. Pine: The Morning he left the city he desired that I wou’d refer the Ladies to you for payment; and you will also please to receive it under your roof. I am Your &ca E. Trist
My Dear Sir London 13 Novr. 1794 I am come home to England, and have the great pleasure to find I am not forgoten by Mr: Jefferson, t’is impossible to express my happiness, the less I say the better, and am Sure what I dont say will be added by a Heart who can conceive and interpatrate Sentiments of a feeling and greatfull heart. My Angelica has been the greatest joy on my return. She has flatterd me much by telling me my name was mentiond in most of the letters which come from America. Mr: Trumbull tells me the Same and offers to send a letter; Now I have Not time to make One, till now I did Not know how to send one, but hope that I shall in another find more to convince You how much I am your Most Affte. Maria Cosway
Dear Sir London 24 Novr: 1794 I sent you a very short letter On My Arrival here, but promised Soon a longer One, here I am ready for it, from a great wish to converse a little with One whom ought to be my freind from a simpathy of Sentiments: You know this does not Mean that the reasons are the same, Mine may  be raised by the Consciousness of your Merits, in that case I have nothing to pretend from you, but you May then return some acknowledgement built only on a sort of gratitude that will mingle the tenderness of freindship, forgit from what source it Comes, and be pure in its effect. Mr: Trumbull told me a few days ago he had an opportunity of sending a Letter, but was prevented from sending to him One. Last Night Mrs: Church told me she has another; sure t’is Opportunities coming with a reproach when they are felt with so much pleasure, and at the same time as a reward for what Seems a negligence not embracing the first without a fault, a second recompences the sensearly felt loss. Now this will Come accompanied by One from the Most charming of woman, My Angelica, I love her so much that I think and am persuaded she must be beloved by every One who know her, therefore give value to every thing which Comes from her Or she Notices with her regard. I will think she has Some attachment for me and I value it much. My great fear is that soon I shall loos her, I even thought I should not find her in England, but have been fortunate to meet this pleasure On my arrival, and certainly she was a great consolation to me. Could I but think it a lasting One, You know this Country, and believe you have heard My sentiments on it, My long stay in Italy and particularly the fine Climate and Most beautifull situation of Genova has Not alterd them but increased a surmontable Antipathy I feel, though the pleasure of the good society and amiable freinds Make in great Measure a recompence, t’is accompanied with a dampness of a gloomy cloud which withers the first blossom of the appearing charm, waits for some glimps of the raising sunn, and stops till t’is forgot in thought in Amazement, or indiference. I often think of America, and every thing I hear of it pleases me and Makes me wish to come, why Can I not come? How Many thinks are like the Italians and Italy? There is a Comfort in freinds and their Society. I have found a preaty little girl, I hope she will make some Comfort, she shows Natural Talent and a good, Soft disposition—painting and Musik for the present are forgoten by me, the long neglect has made me now give them up, and find no loss, better occupations will fill up My heavy hours. Je vous fait Mon Compliment, et je vous suppose deja un gran Papa! May you have in every Circumstance of your life that happiness you so Much deserve, Much for the choise you Make of your happiness. Oh how few at this Moment! Where is there a smal spot unknown to Misery, trouble, and Confusion? You have retierd from it, but Much is the loss for those you Might have been of use to. Mr: Cosway desires his Compliments and joins with me in all I have said. He Might have wrote  better English, but My wishes I will not give up to any body. Remember Me ever as one of your Most affte: freinds Maria Cosway
Sir Fredericksburg June 30th 95— I had the pleasure of receiving your letter of the 4th. instant and should have answered it some-time since, but have been for several weeks a good deal indisposed by the small pox, a circumstance, which, I hope, will apologize for the delay. I have no objection to the establishment of the line by which my son Champe had devided the lands sold you and there held by me as I am fully perswaided that he would do nothing to affect my interest. I must beg that you will accept my acknowledgements for your polite communication and beleive me to be With respect yr. Most Obt Sally Carter
Dear Sir London 4 Decr: 1795 At last I have the long wished pleasure of receiving a letter from you? I cannot tell how much it has made me happy for I could not suspect you could forgit me, tho am sensible My not having sufficient Merit to engage your remembrance, but can only trust to the Sentiments known to me for so long a time and formed Upon So much Sure foundation. How glad am I to hear your detachment from the busling world, what is this world? Happy, very happy those who Make it a good passage to a better One, to an everlasting life, we can be very happy in this with these veiws of it for then we only think of the virtues we can practice, the vices we Can avoid, the end we are created for, the recompence is destind to us which no body can usurpe and when we act for the Omnipresent or Omnipotent who sees every thing how can we wish, or be disturb’d at the Momentary circumstances which pass like lightning to leave every thing in ashes. You will soon have the pleasure of seeing the Charming Anjelica. I loose her with Much regret she is the woman I love Most, and feel Most happy with in this Country. Poor Madm. de Corney has met with a great Change in her life from what she was. I wanted Much to send a letter to you by My brother George Hadfield but the resolution of his going away without taking leave prevented My knowing of his departure till he was gone, however I know You will be kind to all and need Not Say any thing particular for him. I hope he will meet with encouragement, he has talents and an amiable Character, tho he is my brother I must be just. You Mention Geneva in Your letter with satisfaction. Many thanks for I love it very Much as well as admire it. How happy I was there in those enchanting walls of the Most beautifull Situation. What a sound a Monastory is‡ If all thought it the heaven I do, the world would end and the gospel obliged to alter its prediction. What a differance in this **town** in this bustling situation, from that solitude? Those bells who put me in Mind Constantly of our Maker our duties and our end, who united My Mind with angels and heaven‡ Here indeed the Crowns must be treeple and so they will be if recompance is to be according to difficulty and pain. My duties however are here and here I make My Crowns, my happiness in the will of God, a will which Must be done, let us follow it with Love. I will not attempt to enter into political news or insignificant things. Theathers and Masquerades, assemblies Concerts and Cards, Shops and shows make all the occupation of these good people. My little girl,  my pencil and my home make mine, and endevor to make my time very short by making it useful. What would I give to surprize you on your Monticello! I have Your picture by Troumbel on the side of my Chimney always before me, and always regret that perhaps never can I see the Original. At least if I could have oftner your letters t’would be some compensation, but to be deprived of both is too much. Resolve and break this long intervals. Yours most affy: M. Cosway
Varina Feb. 27th. 1796 [i.e. 1797] We arrived here, Dear Papa, last thursday without any accident and found my sister and her children in perfect health; she enjoying the satisfaction arising from the consciousness of fulfilling her duty to the utmost extent. But it is one she has always had. It would please you, I am sure, to see what an economist, what a manager she is become. The more I see of her the more I am sensible how much more deserving she is of you than I am, but my dear Papa suffer me to tell you that the love, the gratitude she has for you can never surpass mine; it would not be possible. * * * Adieu, dear Papa. It is one o’clock. The letters go off to-morrow by daylight and I only knew this night that I could write to you or I should have written to Sallie Cropper. Adieu dear Papa; believe me your most affectionate child

